## Download Sample garak report for gpt35-0906.report.jsonl

In [17]:
!pip uninstall tensorflow sentence-transformers

^C


In [ ]:

!pip install sentence-transformers torch pandas scikit-learn joblib requests

  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl (375.7 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
  Rolling back uninstall of tensorflow
  Moving to c:\users\dell\anaconda3\envs\nlp\lib\site-packages\tensorflow-2.18.0.dist-info\
   from C:\Users\DELL\anaconda3\envs\nlp\Lib\site-packages\~ensorflow-2.18.0.dist-info


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\DELL\\anaconda3\\envs\\nlp\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
#!/usr/bin/env python3
import json
import csv
import os
import urllib.request

# Download sample Garak report if not present
url = "https://gist.githubusercontent.com/shubhobm/9fa52d71c8bb36bfb888eee2ba3d18f2/raw/ef1808e6d3b26002d9b046e6c120d438adf49008/gpt35-0906.report.jsonl"
input_file = "garak.report.jsonl"
if not os.path.exists(input_file):
    print("Downloading sample Garak report...")
    urllib.request.urlretrieve(url, input_file)
    print("✅ Downloaded:", input_file)

output_file = "garak_report_flat.csv"

# Status decoding
def parse_status(status_code):
    if status_code == 1:
        return "Pass"
    elif status_code == 2:
        return "Fail"
    else:
        return "Not Evaluated"

# Parse turns-based or prompt-based format
def extract_input_output(record):
    turns = record.get("notes", {}).get("turns", [])
    if turns:  # Multi-turn conversation
        attacker, bot = [], []
        for role, msg in turns:
            msg = msg.strip().replace("\n", " ")
            if role == "probe":
                attacker.append(msg)
            elif role == "model":
                bot.append(msg)
        return " | ".join(attacker), " | ".join(bot)

    # Fallback to single-turn prompt + outputs
    prompt = record.get("prompt", "").strip().replace("\n", " ")
    outputs = record.get("outputs", [])
    output_texts = [o.strip().replace("\n", " ") for o in outputs]
    return prompt, " | ".join(output_texts)

# Main conversion loop
with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", newline='', encoding="utf-8") as outfile:
    writer = csv.DictWriter(outfile, fieldnames=[
        "uuid", "probe_classname", "attacker_input", "target_bot_response", "status", "goal", "trigger"
    ])
    writer.writeheader()

    for line in infile:
        record = json.loads(line)
        if record.get("entry_type") != "attempt":
            continue

        uuid = record.get("uuid", "")
        probe_class = record.get("probe_classname", "")
        status = parse_status(record.get("status"))
        attacker_input, bot_response = extract_input_output(record)
        goal = record.get("goal", "")
        trigger = record.get("notes", {}).get("trigger", "")

        writer.writerow({
            "uuid": uuid,
            "probe_classname": probe_class,
            "attacker_input": attacker_input,
            "target_bot_response": bot_response,
            "status": status,
            "goal": goal,
            "trigger": trigger
        })

print(f"✅ Garak report successfully converted to: {output_file}")


## Download Latest cves data from NVD to train own CVSS seviarity score kind of model 

In [ ]:
import requests
import gzip
import json
from pathlib import Path

# Define URL for latest CVE data (as of 2025)
url = "https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2025.json.gz"

# Define download path
download_path = Path("nvdcve-1.1-2025.json.gz")

# Step 1: Download the gzip file
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded: {download_path}")
else:
    raise Exception(f"Failed to download file: HTTP {response.status_code}")

# Step 2: Extract and parse JSON
with gzip.open(download_path, 'rt', encoding='utf-8') as f:
    cve_data = json.load(f)

# Step 3: Preview structure
cve_items = cve_data.get("CVE_Items", [])

# Step 4: Extract CVSS-related information from each CVE item
parsed_cve_list = []
for item in cve_items:
    cve_id = item["cve"]["CVE_data_meta"]["ID"]
    description_data = item["cve"]["description"]["description_data"]
    description = next((d["value"] for d in description_data if d["lang"] == "en"), "")

    impact = item.get("impact", {})
    base_metric_v3 = impact.get("baseMetricV3", {})
    cvss_v3 = base_metric_v3.get("cvssV3", {})

    base_score = cvss_v3.get("baseScore", None)
    severity = cvss_v3.get("baseSeverity", None)
    vector_string = cvss_v3.get("vectorString", None)

    parsed_cve_list.append({
        "CVE_ID": cve_id,
        "Description": description,
        "BaseScore": base_score,
        "Severity": severity,
        "Vector": vector_string
    })

# Output sample of parsed data
parsed_cve_list[:5]  # Show first 5 entries as preview


## Training Custom Model

In [16]:
import pandas as pd
import json
import gzip
import os
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# ----------------------------------------
# STEP 1: Load and preprocess NVD data
# ----------------------------------------

def load_nvd_data(filepath):
    with gzip.open(filepath, 'rt', encoding='utf-8') as f:
        data = json.load(f)
    records = []
    for item in data.get('CVE_Items', []):
        try:
            desc = item['cve']['description']['description_data'][0]['value']
            impact = item.get('impact', {})
            base_metric = impact.get('baseMetricV3') or impact.get('baseMetricV2')
            if base_metric:
                severity = base_metric.get('cvssV3', {}).get('baseSeverity') or base_metric.get('severity')
                if severity and desc:
                    records.append((desc.strip(), severity.strip().capitalize()))
        except Exception:
            continue
    return pd.DataFrame(records, columns=["description", "severity"])

# ----------------------------------------
# STEP 2: Train model
# ----------------------------------------

def train_cvss_classifier(df):
    print(f"Loaded {len(df)} NVD entries.")

    # Encode labels
    le = LabelEncoder()
    y = le.fit_transform(df['severity'])

    # Embedding
    model = SentenceTransformer("all-MiniLM-L6-v2")
    print("Encoding NVD descriptions...")
    X = model.encode(df['description'].tolist(), show_progress_bar=True)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

    # Save everything
    joblib.dump(clf, "trained_cvss_classifier.pkl")
    joblib.dump(le, "cvss_label_encoder.pkl")
    return model, clf, le

# ----------------------------------------
# STEP 3: Predict on Garak report
# ----------------------------------------

def predict_on_garak(model, clf, le, garak_path, output_path):
    df = pd.read_csv(garak_path)
    df["full_text"] = df["attacker_input"].fillna('') + " " + df["bot_response"].fillna('')
    print("Embedding Garak report...")
    embeddings = model.encode(df["full_text"].tolist(), show_progress_bar=True)

    y_pred = clf.predict(embeddings)
    df["predicted_severity"] = le.inverse_transform(y_pred)
    df.to_csv(output_path, index=False)
    print(f"✅ Predictions saved to {output_path}")

# ----------------------------------------
# MAIN EXECUTION
# ----------------------------------------

if __name__ == "__main__":
    # Download NVD data if not present
    nvd_file = "nvdcve-1.1-2025.json.gz"
    if not os.path.exists(nvd_file):
        import requests
        print("Downloading latest NVD data...")
        url = f"https://nvd.nist.gov/feeds/json/cve/1.1/{nvd_file}"
        response = requests.get(url)
        with open(nvd_file, "wb") as f:
            f.write(response.content)

    # Train on NVD
    nvd_df = load_nvd_data(nvd_file)
    embed_model, classifier, label_encoder = train_cvss_classifier(nvd_df)

    # Predict on Garak report
    predict_on_garak(
        model=embed_model,
        clf=classifier,
        le=label_encoder,
        garak_path="garak_report_flat.csv",
        output_path="garak_with_severity.csv"
    )


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.